In [59]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

X=pd.read_csv('../data/processed/X.csv')
Y=pd.read_csv('../data/processed/Y.csv')
X_predict=pd.read_csv('../data/processed/X_predict.csv')
df_predict=pd.read_csv('../data/processed/df_predict.csv')

In [60]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,28 mins 38 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_carpiero_5g7hq4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.344 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [61]:
X_Y=pd.concat([X,Y], axis=1)


In [62]:
x= X.columns.to_list()
y = "price"

In [63]:
train_X_Y=h2o.H2OFrame(X_Y)
aml = H2OAutoML(max_models=100, max_runtime_secs= 600, seed= 32,sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_X_Y)


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [64]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200908_194059,513.971,264167,264167,260.895,0.0898133
StackedEnsemble_BestOfFamily_AutoML_20200908_194059,517.982,268305,268305,265.102,0.0935735
GBM_3_AutoML_20200908_194059,520.619,271044,271044,267.47,0.0934392
GBM_4_AutoML_20200908_194059,521.61,272077,272077,263.87,0.08861
GBM_2_AutoML_20200908_194059,525.515,276166,276166,272.697,0.0981969
GBM_1_AutoML_20200908_194059,526.061,276740,276740,274.816,0.100369
GBM_grid__1_AutoML_20200908_194059_model_6,528.418,279226,279226,266.354,0.0889
GBM_grid__1_AutoML_20200908_194059_model_3,530.39,281313,281313,276.785,0.101318
GBM_grid__1_AutoML_20200908_194059_model_1,540.989,292669,292669,268.628,0.0891584
GBM_grid__1_AutoML_20200908_194059_model_4,545.334,297389,297389,276.203,0.0930006


In [65]:
predict_X=h2o.H2OFrame(X_predict)
predictions_competition= aml.leader.predict(predict_X)
predictions_competition=predictions_competition.as_data_frame()


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [66]:

submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition['predict']})
submission['id'] = submission['id'].astype('int64')

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)



,id,price
0,0,2925.883844
1,1,5683.858634
2,2,9688.156861
3,3,3976.050817
4,4,1599.480311
...,...,...
13480,13480,1680.821144
13481,13481,2360.353642
13482,13482,3024.134867
13483,13483,2115.672773


In [71]:
h2o.cluster().shutdown()

In [69]:
submission.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,3925.832887
std,3892.928525,3948.964209
min,0.000000,324.060421
25%,3371.000000,913.215152
50%,6742.000000,2430.916861
75%,10113.000000,5298.220727
max,13484.000000,18362.088564
